In [1]:
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import requests
from io import BytesIO
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
from skimage.transform import rescale, resize, downscale_local_mean
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob
import random
import shutil

In [ ]:
# resize image
gts = glob.glob("Ovale/gt/*")

for gt in gts:
    print(gt)
    img = imread(gt, plugin='pil')
    new_img = resize(img,(1200,1600),anti_aliasing=True)
    new_name = gt.split(".")[0] + ".jpg"
    imsave(new_name,new_img)
    print(new_name)

print("--------img start---------") 
gts = glob.glob("Ovale/img/*")
for gt in gts:
    print(gt)
    img = imread(gt, plugin='pil')
    new_img = resize(img,(1200,1600),anti_aliasing=True)
    new_name = gt.split(".")[0] + ".jpg"
    imsave(new_name,new_img)
    print(new_name)

In [ ]:
class_id = "1"
count = 1
gts = glob.glob("Ovale/gt/*.jpg")
main_img_pth = "Ovale/img/"
for gt in gts:
    gt_image = gt.split("/")[-1]
    txt_name = "po-{:010d}.txt".format(int(count))
    img_name = "po-{:010d}.jpg".format(int(count))
    print(gt_image)
    img_main = imread(main_img_pth + gt_image)
    imsave(os.path.join("tmp_images/",img_name),img_main)
    img = imread(gt)
    h, w = img.shape
    thresh = threshold_otsu(img)
    bw = img > thresh
    label_image = label(bw)
    result = ""
    for region in regionprops(label_image):
        minr, minc, maxr, maxc = region.bbox
        x1 = minc / w
        y1 = minr / h
        bw = (maxc - minc) / w
        bh = (maxr - minr) / h
        xc = x1 + (bw / 2)
        yc = y1 + (bh / 2)
        result += "\n" + class_id + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh) 
    f = open("tmp_labels/" + txt_name,'w')
    result = result[1:]
    f.write(result)
    f.close()
    count += 1

In [2]:
count = 30
files = glob.glob("tmp_labels/*.txt")
degs = [45,90,135,180,225,270,315]
for file in files:
    print(file)
    f = open(file,"r")
    lines = f.readlines()
    f.close()
    rects = []
    img_name = file.split("/")[1].replace(".txt",".jpg")
    for line in lines:
        class_id, xc, yc, bw, bh = line.split(" ")
        xc = float(xc)
        yc = float(yc)
        bw = float(bw)
        bh = float(bh)
        minx = (xc - (bw/2)) * 1600
        miny = (yc - (bh/2)) * 1200
        bw = bw * 1600
        bh = bh * 1200
        maxx = minx + bw
        maxy = miny + bh
        rects.append(BoundingBox(x1=int(minx),y1=int(miny),x2=int(maxx),y2=int(maxy),label=class_id))
    img = Image.open("tmp_images/"+img_name)
    img = np.array(img)
    bbs = BoundingBoxesOnImage(rects,shape=img.shape)
    for deg in degs:
        name = "po-{:010d}".format(count)
        image_aug, bbs_aug = iaa.Affine(rotate=deg,mode="reflect")(image=img, bounding_boxes=bbs)
        aug_h, aug_w, aug_c = image_aug.shape
        clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=False).clip_out_of_image()
        bboxs = ""
        for bb in clip_off_bbs.bounding_boxes:
            x1 = bb.x1
            y1 = bb.y1
            x2 = bb.x2
            y2 = bb.y2
            bb_w = (x2-x1)/aug_w
            bb_h = (y2-y1)/aug_h
            bb_xc = (x1/aug_w) + (bb_w/2)
            bb_yc = (y1/aug_h) + (bb_h/2)
            bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
        bboxs = bboxs[1:]
        im_aug = Image.fromarray(image_aug)
        im_aug.save(os.path.join('tmp_images',name+".jpg"))
        f = open(os.path.join('tmp_labels',name+".txt"),'w')
        f.write(bboxs)
        f.close()
        print("{} {}".format(count,deg))
        count += 1

tmp_labels/po-0000000001.txt
30 45
31 90
32 135
33 180
34 225
35 270
36 315
tmp_labels/po-0000000002.txt
37 45
38 90
39 135
40 180
41 225
42 270
43 315
tmp_labels/po-0000000003.txt
44 45
45 90
46 135
47 180
48 225
49 270
50 315
tmp_labels/po-0000000004.txt
51 45
52 90
53 135
54 180
55 225
56 270
57 315
tmp_labels/po-0000000005.txt
58 45
59 90
60 135
61 180
62 225
63 270
64 315
tmp_labels/po-0000000006.txt
65 45
66 90
67 135
68 180
69 225
70 270
71 315
tmp_labels/po-0000000007.txt
72 45
73 90
74 135
75 180
76 225
77 270
78 315
tmp_labels/po-0000000008.txt
79 45
80 90
81 135
82 180
83 225
84 270
85 315
tmp_labels/po-0000000009.txt
86 45
87 90
88 135
89 180
90 225
91 270
92 315
tmp_labels/po-0000000010.txt
93 45
94 90
95 135
96 180
97 225
98 270
99 315
tmp_labels/po-0000000011.txt
100 45
101 90
102 135
103 180
104 225
105 270
106 315
tmp_labels/po-0000000012.txt
107 45
108 90
109 135
110 180
111 225
112 270
113 315
tmp_labels/po-0000000013.txt
114 45
115 90
116 135
117 180
118 225
119 270

In [3]:
# create test_image
images = glob.glob("tmp_images/*.jpg")
for image in images:
    img = Image.open(image)
    weigth,height = img.size
    print(weigth)
    txt_name = image.replace("tmp_images/","tmp_labels/").replace(".jpg",".txt")
    f = open(txt_name,"r")
    bboxs = list(f)
    f.close()
    for box in bboxs:
        coor = box.split()
        label = int(coor[0])
        xc = float(coor[1])*weigth
        yc = float(coor[2])*height
        bb_w = float(coor[3])*weigth
        bb_h = float(coor[4])*height
        x1 = xc-(bb_w/2)
        y1 = yc-(bb_h/2)
        x2 = x1 + bb_w
        y2 = y1 + bb_h
        draw = ImageDraw.Draw(img)
        draw.rectangle((x1, y1,x2,y2), fill=None,outline="red",width=5)
    save_path = image.replace("tmp_images/","test_images/")
    img.save(save_path)
    print(save_path)

1600
test_images/po-0000000001.jpg
1600
test_images/po-0000000002.jpg
1600
test_images/po-0000000003.jpg
1600
test_images/po-0000000004.jpg
1600
test_images/po-0000000005.jpg
1600
test_images/po-0000000006.jpg
1600
test_images/po-0000000007.jpg
1600
test_images/po-0000000008.jpg
1600
test_images/po-0000000009.jpg
1600
test_images/po-0000000010.jpg
1600
test_images/po-0000000011.jpg
1600
test_images/po-0000000012.jpg
1600
test_images/po-0000000013.jpg
1600
test_images/po-0000000014.jpg
1600
test_images/po-0000000015.jpg
1600
test_images/po-0000000016.jpg
1600
test_images/po-0000000017.jpg
1600
test_images/po-0000000018.jpg
1600
test_images/po-0000000019.jpg
1600
test_images/po-0000000020.jpg
1600
test_images/po-0000000021.jpg
1600
test_images/po-0000000022.jpg
1600
test_images/po-0000000023.jpg
1600
test_images/po-0000000024.jpg
1600
test_images/po-0000000025.jpg
1600
test_images/po-0000000026.jpg
1600
test_images/po-0000000027.jpg
1600
test_images/po-0000000028.jpg
1600
test_images/po-

In [4]:
# gen url
count = 1
for i in range(count,233):
    name = "po-{:010d}.jpg".format(i)
    print("https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/"+name)

https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/po-0000000001.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/po-0000000002.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/po-0000000003.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/po-0000000004.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/po-0000000005.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/po-0000000006.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/po-0000000007.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/po-0000000008.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/po-0000000009.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/po-0000000010.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Ovale/po-0000000011.jpg
https://raw.githubusercontent.com/RTANC/mal

In [ ]:
count = 15250
for i in range(count,15367):
    name = "{:010d}".format(i)
    print(name)
    os.remove(os.path.join("images/",name+".jpg"))
    os.remove(os.path.join("labels/",name+".txt"))